# Import

In [51]:
# 平行
import multiprocessing as mp
from functools import partial

# 進度條
from tqdm import tqdm
import time
from datetime import datetime, timedelta

In [52]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns


In [53]:
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from typing import Dict, Tuple
import torch

# Data

In [54]:
# Feature dictionary(fundamental feature)

path = r"C:\Users\User\Desktop\quant\Sector Allocation Framework under Reinforcement Learning Rotation\industry-rotation\industryRL_v5\pmart.parquet"
df = pd.read_parquet(path)
pmart_column_lst = df.columns.tolist()[2:]
pmart_data_dict = {}

pmart_column_lst = df.columns.tolist()[2:]
pmart_column_lst = [col for col in pmart_column_lst if col not in ['季別', '合併(Y/N)', '單季(Q)/單半年(H)', '月份', '幣別', '近一週報酬率(%)', '近一月報酬率(%)', '近一季報酬率(%)', '近一年報酬率(%)']]

for column in pmart_column_lst:
    temp_df = df[['datetime', 'symbol' , column]]
    pmart_data_dict[column] = temp_df.pivot_table(
    index='datetime',  # 行索引
    columns='symbol',  # 列名稱
    values=column,  # 數值
)
    
industry_path = r"C:\Users\User\Desktop\quant\Sector Allocation Framework under Reinforcement Learning Rotation\industry-rotation\security_properties.parquet"
property_data = pd.read_parquet(industry_path)

property_data = property_data.reset_index()
property_data = property_data[property_data['industry'].notna()]
symbol_to_industry = property_data[['symbol', 'industry']].set_index('symbol')['industry']

# 創建一個新 dictionary，用於儲存轉換後的 DataFrame
industry_data_dict = {}

for key, df in pmart_data_dict.items():
    # 重設 DataFrame 的列索引，將 symbol 映射為 industry
    df = df.rename(columns=symbol_to_industry).copy()
    
    # 基於 industry 聚合，計算平均值
    df = df.groupby(axis=1, level=0).mean()
    
    # 儲存到新的 dictionary
    industry_data_dict[key] = df

# 結果：industry_data_dict 包含的 DataFrame 是基於產業的全平均

feature_dict = pd.DataFrame()

# 合併多個df以符合RL所需格式
def reorganize_industry_data(industry_data_dict):
    """
    重組產業指標數據，將產業作為第一層，指標作為第二層
    
    Parameters:
    industry_data_dict: Dict[str, pd.DataFrame] - 指標資料字典，key為指標名稱
    
    Returns:
    pd.DataFrame - 重組後的DataFrame，MultiIndex為(產業, 指標)
    """
    # 取得所有日期作為索引
    all_dates = set()
    for df in industry_data_dict.values():
        all_dates.update(df.index)
    all_dates = sorted(list(all_dates))
    
    # 取得所有產業名稱
    industries = industry_data_dict[list(industry_data_dict.keys())[0]].columns
    
    # 創建新的DataFrame
    result_dict = {}
    
    # 重組數據
    for industry in industries:
        for indicator, df in industry_data_dict.items():
            if industry in df.columns:
                # 使用元組作為鍵，第一層是產業，第二層是指標
                result_dict[(industry, indicator)] = df[industry]
    
    # 創建最終的DataFrame
    result_df = pd.DataFrame(result_dict, index=all_dates)
    
    # 確保MultiIndex格式正確
    result_df.columns = pd.MultiIndex.from_tuples(result_df.columns)
    
    return result_df

# Fundamental feature

fundamental_lst = [
    "每股盈餘",
    "ROA(C) 稅前息前折舊前",  # 資產報酬率
    "ROE(A)-稅後",  # 股東權益報酬率
    "營業毛利率",
    "稅後淨利率",
    "CFO/合併總損益",
    "現金流量比率",
    "營收成長率",
    "營業毛利成長率",
]

fundamental_dict = {key: industry_data_dict[key] for key in fundamental_lst if key in industry_data_dict}

fundamental_feature = reorganize_industry_data(fundamental_dict)
fundamental_feature

C:\Users\User\AppData\Local\Temp\ipykernel_40720\446902924.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_40720\446902924.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_40720\446902924.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_40720\446902924.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_40720\446902924.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.

光電業                                                           \
                每股盈餘 ROA(C) 稅前息前折舊前 ROE(A)-稅後     營業毛利率      稅後淨利率     現金流量比率   
2013-05-15  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
2013-05-16  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
2013-05-17  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
2013-05-20  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
2013-05-21  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
...              ...            ...       ...       ...        ...        ...   
2022-12-26  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   
2022-12-27  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   
2022-12-28  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   
2022-12-29  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   
2022-12-30  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   

                                         其他                 ...      電腦及週邊  \
                營收成長率     營業毛利成長率      每股盈餘 ROA(C) 稅前息前折舊前  ...    營業毛利成長率   
2013-05-15   3.346697 -568.294404  0.145172       0.874655  ...  37.108152   
2013-05-16   3.346697 -568.294404  0.145172       0.874655  ...  37.108152   
2013-05-17   3.346697 -568.294404  0.145172       0.874655  ...  37.108152   
2013-05-20   3.346697 -568.294404  0.145172       0.874655  ...  37.108152   
2013-05-21   3.346697 -568.294404  0.145172       0.874655  ...  37.108152   
...               ...         ...       ...            ...  ...        ...   
2022-12-26 -11.598415  -50.415122  0.764286       1.794821  ...  21.390556   
2022-12-27 -11.598415  -50.415122  0.764286       1.794821  ...  21.390556   
2022-12-28 -11.598415  -50.415122  0.764286       1.794821  ...  21.390556   
2022-12-29 -11.598415  -50.415122  0.764286       1.794821  ...  21.390556   
2022-12-30 -11.598415  -50.415122  0.764286       1.794821  ...  21.390556   

              食品工業                                                           \
              每股盈餘 ROA(C) 稅前息前折舊前 ROE(A)-稅後      營業毛利率     稅後淨利率  CFO/合併總損益   
2013-05-15  0.3750       2.375417   1.88125  17.498333 -1.977083  31.411053   
2013-05-16  0.3750       2.375417   1.88125  17.498333 -1.977083  31.411053   
2013-05-17  0.3750       2.375417   1.88125  17.498333 -1.977083  31.411053   
2013-05-20  0.3750       2.375417   1.88125  17.498333 -1.977083  31.411053   
2013-05-21  0.3750       2.375417   1.88125  17.498333 -1.977083  31.411053   
...            ...            ...       ...        ...       ...        ...   
2022-12-26  0.5544       1.968800   1.71160  21.709600  0.131600   2.321176   
2022-12-27  0.5544       1.968800   1.71160  21.709600  0.131600   2.321176   
2022-12-28  0.5544       1.968800   1.71160  21.709600  0.131600   2.321176   
2022-12-29  0.5544       1.968800   1.71160  21.709600  0.131600   2.321176   
2022-12-30  0.5544       1.968800   1.71160  21.709600  0.131600   2.321176   

                                           
               現金流量比率     營收成長率   營業毛利成長率  
2013-05-15  13.037083  5.011667 -27.79625  
2013-05-16  13.037083  5.011667 -27.79625  
2013-05-17  13.037083  5.011667 -27.79625  
2013-05-20  13.037083  5.011667 -27.79625  
2013-05-21  13.037083  5.011667 -27.79625  
...               ...       ...       ...  
2022-12-26  17.601200  8.454000   1.95800  
2022-12-27  17.601200  8.454000   1.95800  
2022-12-28  17.601200  8.454000   1.95800  
2022-12-29  17.601200  8.454000   1.95800  
2022-12-30  17.601200  8.454000   1.95800  

[2367 rows x 204 columns]

In [55]:
# Feature lists
technical_lst = [
    "開盤價-除權息", "最高價-除權息", "最低價-除權息", "收盤價-除權息",
    "成交量(千股)", "交易所本益比", "交易所淨值市價比", "日報酬率(%)"
]

institutional_lst = [
    "外資買賣超(千股)", "投信買賣超(千股)", "自營買賣超(千股)",
    "外資總持股率(%)", "投信總持股率(%)", "自營總持股率(%)"
]

margin_lst = [
    "融資餘額(張)", "融券餘額(張)", "融資使用率", "融券使用率", "券資比"
]

# Create feature dictionaries
technical_dict = {key: industry_data_dict[key] for key in technical_lst if key in industry_data_dict}
institutional_dict = {key: industry_data_dict[key] for key in institutional_lst if key in industry_data_dict}
margin_dict = {key: industry_data_dict[key] for key in margin_lst if key in industry_data_dict}

# Add to feature_dict
price_volume_feature = reorganize_industry_data(technical_dict)
institutional_feature = reorganize_industry_data(institutional_dict)
margin_feature = reorganize_industry_data(margin_dict)

In [56]:
# return data
return_path = r'C:\Users\User\Desktop\quant\Sector Allocation Framework under Reinforcement Learning Rotation\industry-rotation\industryRL_v3\returns_data.csv'
return_data = pd.read_csv(return_path, index_col=0)
return_data 

,光電業,其他,化學工業,半導體,居家生活,建材營造,數位雲端,文化創意業,材料,汽車工業,...,資訊服務業,通信網路業,金融業,鋼鐵工業,電子商務,電子通路業,電子零組件,電機機械,電腦及週邊,食品工業
datetime,,,,,,,,,,,,,,,,,,,,,
2013-05-15,1.302608,0.642286,0.555994,0.421326,1.430424,1.199804,0.832743,0.068016,0.975402,1.106733,...,0.286967,0.635315,1.274580,0.236545,-0.3968,-0.102849,0.512790,0.240838,0.603218,0.727992
2013-05-16,1.185714,-0.036057,0.197428,0.836739,0.817339,0.699168,2.735586,0.446837,0.767758,1.139428,...,-0.042337,0.187807,2.271045,0.364575,1.1952,0.554331,0.188547,0.507773,0.340185,0.243037
2013-05-17,1.512616,0.441230,0.179203,0.155004,0.421407,0.382724,3.806300,0.589184,0.117923,0.165011,...,-0.043563,0.443001,-0.204555,0.104760,6.6929,0.521608,0.628443,0.076025,0.577626,0.914350
2013-05-20,1.834277,0.620290,0.049575,0.013956,0.202222,0.684601,1.009814,-0.270089,0.459767,0.013900,...,0.825748,0.479855,0.919307,0.421353,-4.7970,0.774603,0.669102,0.433365,0.482416,0.439271
2013-05-21,1.433981,0.840786,0.208633,1.255811,0.770324,1.245729,0.011843,-0.419305,0.657993,0.488272,...,1.186656,0.174277,-0.196455,1.180495,-0.7752,1.235131,0.763970,0.588637,0.942956,0.161783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-13,0.902497,0.182257,0.061393,1.664595,0.060207,-0.052452,0.783233,0.289356,-0.222916,0.072225,...,0.321056,0.448566,0.200614,0.201419,NaN,0.759009,0.579641,0.729533,0.719946,0.022267
2024-06-14,1.033071,0.509263,-0.049464,0.548502,0.170418,1.386225,-0.216948,0.644084,0.523718,0.100047,...,0.002290,0.460051,0.164769,0.224991,NaN,0.264346,0.740764,1.100432,0.616124,0.269683
2024-06-17,0.965813,0.409280,0.470198,0.795250,0.170254,-1.090854,0.241833,1.275944,0.419556,0.200936,...,0.059705,1.048089,0.114348,0.559096,NaN,0.750651,0.612861,0.754979,0.619787,0.263377


In [57]:
feature_dict = {}
feature_dict['fundamantal'] = fundamental_feature
feature_dict['price_volume'] = price_volume_feature
feature_dict['institutional'] = institutional_feature
feature_dict['margin'] = margin_feature


# Def

In [72]:
class SimpleIndustryEnv(gym.Env):
    def __init__(self, features_dict: Dict[str, pd.DataFrame], returns_data: pd.DataFrame):
        super().__init__()
        
        self.features_dict = self._process_features_dict(features_dict)
        self.returns_data = returns_data
        self.industry_names = list(returns_data.columns)  # Store column names as list
        self.n_industries = len(self.industry_names)
        self.total_features = sum(df.shape[1] for df in self.features_dict.values())
        
        self.action_space = spaces.Discrete(self.n_industries)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(self.total_features,),
            dtype=np.float32
        )
        
        self.current_step = 0
        
    def _process_features_dict(self, features_dict):
        processed = {}
        for key, df in features_dict.items():
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = ['_'.join(map(str, col)).strip() for col in df.columns]
            processed[key] = df.fillna(0)
        return processed
    
    def _get_state(self) -> np.ndarray:
        current_date = self.returns_data.index[self.current_step]
        state = []
        
        for df in self.features_dict.values():
            values = df.loc[current_date].values
            values = np.nan_to_num(values, nan=0.0)
            state.extend(values)
            
        return np.array(state, dtype=np.float32)
    
    def step(self, action: int) -> Tuple[np.ndarray, float, bool, bool, dict]:
        next_returns = self.returns_data.iloc[self.current_step + 1]
        reward = float(next_returns.iloc[action])
        reward = np.nan_to_num(reward, nan=0.0)
        
        self.current_step += 1
        done = self.current_step >= len(self.returns_data) - 2
        
        return self._get_state(), reward, done, False, {}
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        return self._get_state(), {}
    
def calculate_cumulative_returns(returns_data: pd.DataFrame, start_date, end_date) -> pd.Series:
    """Calculate cumulative returns for each industry during the specified period."""
    period_returns = returns_data[start_date:end_date]
    return period_returns.sum()


# Helper function to check data quality
def check_data_quality(features_dict, returns_data):
    print("Returns Data Shape:", returns_data.shape)
    print("\nReturns Data NaN Count:\n", returns_data.isna().sum())
    
    print("\nFeatures Data:")
    for key, df in features_dict.items():
        print(f"\n{key} Shape:", df.shape)
        print(f"{key} NaN Count:\n", df.isna().sum())

In [73]:
def filter_ecommerce(features_dict, returns_data):
    # 過濾 returns_data
    non_ecommerce_returns = returns_data.drop(columns=[col for col in returns_data.columns 
                                                     if '電子商務' in col])
    
    # 過濾 features_dict
    non_ecommerce_features = {}
    for key, df in features_dict.items():
        # 移除包含電子商務的欄位
        filtered_cols = [col for col in df.columns if '電子商務' not in col]
        non_ecommerce_features[key] = df[filtered_cols]
    
    return non_ecommerce_features, non_ecommerce_returns

In [74]:
def train_and_predict(features_dict, returns_data, train_size=120, total_timesteps=10000):
    """單一視窗的訓練和預測函數"""
    try:
        returns_data = returns_data.fillna(0)
        train_features = {k: v.iloc[:train_size].copy() for k, v in features_dict.items()}
        train_returns = returns_data.iloc[:train_size].copy()
        test_returns = returns_data.iloc[train_size:].copy()

        # 計算訓練期間的累積報酬來決定排名
        train_cumulative_returns = train_returns.sum()
        top_industry_indices = train_cumulative_returns.sort_values(ascending=False).head(5).index

        # 計算預測期間的累積報酬
        test_returns_sum = test_returns.sum()

        # 訓練模型
        env = SimpleIndustryEnv(train_features, train_returns)
        model = PPO('MlpPolicy', env, verbose=0, device='cpu', policy_kwargs={'net_arch': [64, 64]})
        model.learn(total_timesteps=total_timesteps)

        result_df = pd.DataFrame({
            'train_start': train_returns.index[0],
            'train_end': train_returns.index[-1],
            'predict_start': test_returns.index[0],
            'predict_end': test_returns.index[-1],
            'rank1_industry': top_industry_indices[0],
            'rank1_return': float(test_returns_sum[top_industry_indices[0]]),
            'rank2_industry': top_industry_indices[1],
            'rank2_return': float(test_returns_sum[top_industry_indices[1]]),
            'rank3_industry': top_industry_indices[2],
            'rank3_return': float(test_returns_sum[top_industry_indices[2]]),
            'rank4_industry': top_industry_indices[3],
            'rank4_return': float(test_returns_sum[top_industry_indices[3]]),
            'rank5_industry': top_industry_indices[4],
            'rank5_return': float(test_returns_sum[top_industry_indices[4]]),
            'actual_best_industry': test_returns_sum.idxmax(),
            'best_return': float(test_returns_sum.max())
        }, index=[0])
        
        return result_df
        
    except Exception as e:
        print(f"\n訓練過程發生錯誤: {str(e)}")
        return None


def train_single_window(window_data, train_size, total_timesteps):
    """單一視窗的訓練函數"""
    try:
        start_idx, features_dict, returns_data = window_data
        
        # 選取當前視窗的訓練資料和預測區間
        current_features = {k: v.iloc[start_idx:start_idx + train_size] for k, v in features_dict.items()}
        current_returns = returns_data.iloc[start_idx:start_idx + train_size + 30]
        
        # 訓練和預測
        result_df = train_and_predict(
            current_features,
            current_returns,
            train_size=train_size,
            total_timesteps=total_timesteps
        )
        
        if result_df is not None:
            # 添加視窗資訊
            result_df['window_number'] = start_idx // 30
            result_df['window_start'] = start_idx
            result_df['window_end'] = start_idx + train_size
            return result_df
        return None
        
    except Exception as e:
        print(f"\n視窗 {start_idx} 訓練發生錯誤: {str(e)}")
        return None

def parallel_rolling_train_predict(features_dict, returns_data, train_size=120, predict_size=30, 
                                 total_timesteps=10000, n_processes=None):
    """使用多進程實現平行視窗訓練和預測"""
    try:
        total_rows = len(returns_data)
        n_iterations = (total_rows - train_size - predict_size) // predict_size + 1
        
        print(f"\n開始執行總共 {n_iterations} 次預測訓練")
        print(f"使用進程數: {n_processes if n_processes else '自動'}")
        print("-" * 50)
        
        # 準備所有視窗的資料
        window_data = [
            (i * predict_size, features_dict, returns_data)
            for i in range(n_iterations)
        ]
        
        # 設定進程數，如果未指定則使用 CPU 核心數量
        if n_processes is None:
            n_processes = mp.cpu_count()
        
        # 建立部分函數以固定其他參數
        train_func = partial(train_single_window, 
                           train_size=train_size,
                           total_timesteps=total_timesteps)
        
        # 使用進程池執行訓練，並使用tqdm顯示進度
        results = []
        with mp.Pool(processes=n_processes) as pool:
            # 使用imap並顯示進度
            for i, result in enumerate(pool.imap_unordered(train_func, window_data)):
                if result is not None:
                    results.append(result)
                # 手動更新進度
                print(f"\r完成進度: {i+1}/{n_iterations} ({(i+1)/n_iterations*100:.1f}%)", 
                      end="", flush=True)
        
        print("\n\n訓練完成！")
        
        # 合併所有預測結果
        if results:
            final_predictions = pd.concat(results, ignore_index=True)
            # 根據窗口編號排序
            final_predictions = final_predictions.sort_values('window_number').reset_index(drop=True)
            return final_predictions
        else:
            print("警告：沒有成功的預測結果")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"\n平行訓練過程發生錯誤: {str(e)}")
        return pd.DataFrame()

In [75]:
feature_dict = {}
feature_dict['fundamantal'] = fundamental_feature
feature_dict['price_volume'] = price_volume_feature
feature_dict['institutional'] = institutional_feature
feature_dict['margin'] = margin_feature


In [76]:
feature_dict, return_data = filter_ecommerce(feature_dict, return_data)
# 檢查數據質量
check_data_quality(feature_dict, return_data)

Returns Data Shape: (2700, 24)

Returns Data NaN Count:
 光電業      0
其他       0
化學工業     0
半導體      0
居家生活     0
建材營造     0
數位雲端     0
文化創意業    0
材料       0
汽車工業     0
油電燃氣業    0
生技醫療     0
綠能環保     0
航運業      0
觀光餐旅     0
資訊服務業    0
通信網路業    0
金融業      0
鋼鐵工業     0
電子通路業    0
電子零組件    0
電機機械     0
電腦及週邊    0
食品工業     0
dtype: int64

Features Data:

fundamantal Shape: (2367, 196)
fundamantal NaN Count:
 光電業   每股盈餘              0
      ROA(C) 稅前息前折舊前    0
      ROE(A)-稅後         0
      營業毛利率             0
      稅後淨利率             0
                       ..
食品工業  稅後淨利率             0
      CFO/合併總損益         0
      現金流量比率            0
      營收成長率             0
      營業毛利成長率           0
Length: 196, dtype: int64

price_volume Shape: (2367, 199)
price_volume NaN Count:
 光電業   開盤價-除權息     0
      最高價-除權息     0
      最低價-除權息     0
      收盤價-除權息     0
      成交量(千股)     0
                 ..
食品工業  收盤價-除權息     0
      成交量(千股)     0
      交易所本益比      0
      交易所淨值市價比    0
      日報酬率(%)     0
Leng

# Training

In [77]:
feature_dict

{'fundamantal':                  光電業                                                           \
                 每股盈餘 ROA(C) 稅前息前折舊前 ROE(A)-稅後     營業毛利率      稅後淨利率     現金流量比率   
 2013-05-15  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
 2013-05-16  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
 2013-05-17  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
 2013-05-20  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
 2013-05-21  0.227706       1.739083 -0.308349   9.02000  -7.825413   7.279908   
 ...              ...            ...       ...       ...        ...        ...   
 2022-12-26  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   
 2022-12-27  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   
 2022-12-28  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   
 2022-12-29  1.575976       2.956707  2.148415  17.13622  28.474756  13.909268   
 

In [69]:
final = pd.DataFrame()  #to save all  prediction results
feature_dict['fundamantal'] = feature_dict['fundamantal'][16:]
feature_dict['price_volume'] = feature_dict['price_volume'][16:]
feature_dict['institutional'] = feature_dict['institutional'][16:]
feature_dict['margin'] = feature_dict['margin'][16:]
return_data = return_data[16:]
check_data_quality(feature_dict, return_data)

Returns Data Shape: (2700, 24)

Returns Data NaN Count:
 光電業      0
其他       0
化學工業     0
半導體      0
居家生活     0
建材營造     0
數位雲端     0
文化創意業    0
材料       0
汽車工業     0
油電燃氣業    0
生技醫療     0
綠能環保     0
航運業      0
觀光餐旅     0
資訊服務業    0
通信網路業    0
金融業      0
鋼鐵工業     0
電子通路業    0
電子零組件    0
電機機械     0
電腦及週邊    0
食品工業     0
dtype: int64

Features Data:

fundamantal Shape: (2351, 196)
fundamantal NaN Count:
 光電業   每股盈餘              0
      ROA(C) 稅前息前折舊前    0
      ROE(A)-稅後         0
      營業毛利率             0
      稅後淨利率             0
                       ..
食品工業  稅後淨利率             0
      CFO/合併總損益         0
      現金流量比率            0
      營收成長率             0
      營業毛利成長率           0
Length: 196, dtype: int64

price_volume Shape: (2351, 199)
price_volume NaN Count:
 光電業   開盤價-除權息     0
      最高價-除權息     0
      最低價-除權息     0
      收盤價-除權息     0
      成交量(千股)     0
                 ..
食品工業  收盤價-除權息     0
      成交量(千股)     0
      交易所本益比      0
      交易所淨值市價比    0
      日報酬率(%)     0
Leng

In [ ]:
rolling_predictions = parallel_rolling_train_predict(
    feature_dict, 
    return_data,
    train_size=120,
    predict_size=30,
    total_timesteps=10000,
    n_processes=2 
)


開始執行總共 86 次預測訓練
使用進程數: 2
--------------------------------------------------
